In [1]:
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import icarusplot

@@ Importing mplstyle from /Users/jskim/lib/icarusplot//icarus_style.mplstyle


In [2]:
WD = '/Users/jskim/Documents/Neutrino/ICARUS/GUNDAM-1.9.X/work'

In [3]:
np.set_printoptions(precision=4, suppress=True)

In [4]:
# Number of total variable bin
NBins = 8

# Total number of bins from the first to regularize
NBinsToReg = 0
# Bins to add diagonal at the end
NBinsToExtend = NBins - NBinsToReg

# Regularization strength in log_10 (tau)
Strength = 0.00

# Prior for the error; templates becomes 1 +- PriorError 
# PriorError = 1.0
# PriorError = 0.20
# PriorError = 0.50
PriorError = 0.4

# PriorError_LoW = 1.0
# PriorError_HiW = 1.0

# PriorError_LoW = 0.4
# PriorError_HiW = 0.4

PriorError_LoW = 0.4
PriorError_HiW = 0.6



In [5]:
# CovName = f'RegCov_Bins{NBinsToReg}Plus{NBinsToExtend}_StrengthTenToThe{Strength:.2f}_Prior{PriorError:.1f}'
CovName = f'RegCov_Bins{NBinsToReg}Plus{NBinsToExtend}_StrengthTenToThe{Strength:.2f}_PriorLoW{PriorError_LoW:.1f}_PriorHiW{PriorError_HiW:.1f}'

print(CovName)

RegCov_Bins0Plus8_StrengthTenToThe0.00_PriorLoW0.4_PriorHiW0.6


In [6]:
Tau = (10**Strength)
print(f'Tau = {Tau}')

Tau = 1.0


In [7]:
C_reg_Inv = np.zeros((NBins, NBins))

for i_x in range(NBinsToReg):
    for i_y in range(NBinsToReg):
        ThisValue = 0.
        if i_x==i_y:
            if i_x==0:
                ThisValue = 1.
            elif i_x==NBinsToReg-1:
                ThisValue = 1.
            else:
                ThisValue = 2.
        elif abs(i_x-i_y)==1:
            ThisValue = -1.
        C_reg_Inv[i_x][i_y] = ThisValue

# for i_x in range(NBinsToExtend):
#     ii_x = NBinsToReg + i_x
#     C_reg_Inv[ii_x][ii_x] = 1.
print('C_reg_Inv:\n', C_reg_Inv)

C_reg_Inv:
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


In [8]:
C_diag_Inv = np.ndarray((NBins, NBins))

for i_x in range(NBins):
    for i_y in range(NBins):
        ThisValue = 0.
        if i_x==i_y:
            ThisValue = PriorError*PriorError
        C_diag_Inv[i_x][i_y] = ThisValue
print('C_diag_Inv:\n', C_diag_Inv)

C_diag_Inv:
 [[0.16 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.16 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.16 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.16 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.16 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.16 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.16 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.16]]


In [9]:
C_Inv = (10**Strength) * C_reg_Inv + C_diag_Inv

In [10]:
print('C_Inv:\n', C_Inv)

C_Inv:
 [[0.16 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.16 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.16 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.16 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.16 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.16 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.16 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.16]]


In [11]:
C = np.linalg.inv(C_Inv)
print('C:\n', C)

C:
 [[6.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   6.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   6.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   6.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   6.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   6.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   6.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   6.25]]


In [12]:
# Modifying C to have diagnoal=1

C_Corr = np.ndarray((NBins, NBins))

for i_x in range(NBins):
    for i_y in range(NBins):
        if i_x==i_y:
            C_Corr[i_x][i_y] = 1.
        else:
            C_Corr[i_x][i_y] /= np.sqrt(C[i_x][i_x] * C[i_y][i_y])
            # C_Modified[i_x][i_y] = 0.

print('C_Corr:\n', C_Corr)

C_Corr:
 [[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]


In [13]:
C_Renormed = C_Corr * (PriorError*PriorError)

print('C_Renormed:\n', C_Renormed)

C_Renormed:
 [[0.16 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.16 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.16 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.16 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.16 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.16 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.16 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.16]]


In [14]:
C_Renormed_LoW = C_Corr * (PriorError_LoW*PriorError_LoW)
C_Renormed_HiW = C_Corr * (PriorError_HiW*PriorError_HiW)

print(C_Renormed_LoW)
print(C_Renormed_HiW)

[[0.16 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.16 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.16 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.16 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.16 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.16 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.16 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.16]]
[[0.36 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.36 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.36 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.36 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.36 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.36 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.36 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.36]]


In [15]:
import ROOT

NSet = 2

OutputFileName = f'{CovName}_NSet{NSet}.root'
f_out = ROOT.TFile(f'{WD}/Configs_ParameterSet/BackgroundFit/{OutputFileName}', 'RECREATE')

h_Cov_TMat = ROOT.TMatrixDSym(NBins * NSet)

for i_Set in range(NSet):

    global_offset = i_Set * NBins

    for i in range(NBins):
        for j in range(NBins):

            # h_Cov_TMat[i+global_offset][j+global_offset] = C_Renormed[i, j]

            if i_Set==0:
                h_Cov_TMat[i+global_offset][j+global_offset] = C_Renormed_LoW[i, j]
            else:
                h_Cov_TMat[i+global_offset][j+global_offset] = C_Renormed_HiW[i, j]

h_Cov_TMat.SetTol(1E-20)

h_Cov_TMat.Write(f'{CovName}_NSet{NSet}')

f_out.cd()
f_out.Close()